## 6.4. 多输入多输出通道

每个RGB输入图像具有$3 \times h \times w$的形状。我们将这个大小为$3$的轴称为通道（channel）维度

### 6.4.1. 多输入通道



In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K).shape

torch.Size([2, 2])

In [3]:
X.shape, K.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]))

### 6.4.2. 多输出通道

多输出通道并不仅是学习多个单通道的检测器，而是为了共同使用而优化的，可以认为每个通道是对不同特征的相应。

用$c_i$和$c_o$分别表示输入和输出通道的数目，并让$k_h$和$k_w$为卷积核的高度和宽度。

为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为$c_i \times k_h \times k_w$的卷积核张量，这样卷积核的形状是$c_o \times c_i \times k_h \times k_w$。

在互相关运算中，**每个输出通道先获取所有输入通道**，再**以对应该输出通道的卷积核**计算出结果

In [4]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [5]:
# 具有3个输出通道的卷积核
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [8]:
corr2d_multi_in_out(X, K).shape

torch.Size([3, 2, 2])

### 6.4.3. $1 \times 1$卷积层

卷积的本质是**有效提取相邻像素间的相关特征**，而$1 \times 1$卷积显然没有此作用

卷积层的特有能力——在**高度和宽度维度**上，**识别相邻元素间相互作用**的能力

In [9]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [10]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

### 6.4.4. 小结

多输入多输出通道可以用来扩展卷积层的模型。

当以每像素为基础应用时，$1 \times 1$卷积层相当于全连接层

$1 \times 1$卷积层通常用于调整网络层的通道数量和控制模型复杂性

In [ ]:
def corr2d_multi(X, K):
    h, w = K.shape
    outh = X.shape[0] - h + 1
    outw = X.shape[1] - w + 1
    K = K.reshape(-1, 1)
    Y = []
    for i in range(outh):
        for j in range(outw):
            Y.append(X[i:i + h, j:j + w].reshape(-1))
    # 这个完全没想到，感谢@thirty
    Y = torch.stack(Y, 0)
    # 用矩阵乘法表示互相关运算
    res = (torch.matmul(Y, K)).reshape(outh, outw)
    return res

def corr2d_multi_in(X, K):
    """多输入通道"""
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    # return sum(d2l.corr2d(x, k) for x, k in zip(X, K))
    return sum(corr2d_multi(x, k) for x, k in zip(X, K))


def corr2d_multi_in_out(X, K):
    """多输出通道"""
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    # return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)